<a href='https://github.com/SeWonKwon' ><div> <img src ='https://slid-capture.s3.ap-northeast-2.amazonaws.com/public/image_upload/6556674324ed41a289a354258718280d/964e5a8b-75ad-41fc-ae75-0ca66d06fbc7.png' align='left' /> </div></a>


필요한 이전에 정의한 class 들

In [9]:
class Height(object):
    def __init__(self):
        self.height = 0 

class NodeBT(object):
    def __init__(self, value=None, level=1):
        self.value = value
        self.level = level
        self.left = None
        self.right = None
        
    def __repr__(self):
        return "{}".format(self.value)
    
    def _add_next_node(self, value, level_here=2):
        new_node = NodeBT(value, level_here)
        if not self.value:
            self.value = new_node
        elif not self.left:
            self.left = new_node
        elif not self.right:
            self.right = new_node
            
        else:
            # 노드에 왼쪽 오른쪽 자식이 모드 있다면,
            # 왼쪽 자식 노드에 새 노드를 추가한다. 
            # 그래서 예제의 트리가 왼쪽으로 치우쳐 있다. 
            self.left = self.left._add_next_node(value, level_here+1)
        return self
    
    def _search_for_node(self, value):
        # 전위 순회(pre-order)로 값을 찾는다. 
        if self.value == value:
            return self
        
        else:
            found = None
            if self.left:
                found = self.left._search_for_node(value)
            if self.right:
                found = found or self.right._search_for_node(value)
            return found
        
    def _is_leaf(self):
        # 왼쪽, 오른쪽 자식이 모드 없는 노드
        return not self.right and not self.left
    
    def _get_max_height(self):
        # 노드에서 최대 높이를 얻는다. - O(n)
        heightr, heightl = 0, 0
        if self.right:
            heightr = self.right._get_max_height() + 1
        if self.left:
            heightl = self.left._get_max_height() + 1
            
        return max(heightr, heightl)
    
    def _is_balanced(self, height = Height()):
        # 균형 트리인지 확인한다. - O(n)
        lh = Height()
        rh = Height()
        
        if self.value is None:
            return True
        
        l, r = True, True
        if self.left:
            l = self.left._is_balanced(lh)
        if self.right:
            r = self.right._is_balanced(rh)
            
        height.height = max(lh.height, rh.height) + 1
        
        if abs(lh.height - rh.height) <= 1:
            return l and r
        
        return False
    
    def _is_bst(self, left=None, right=None):
        # 이진 탐색 트리인지 확인한다 - O(n)
        if self.value:
            if left and self.value < left:
                return False
            if right and self.value > right:
                return False
            
            l, r = True, True
            if self.left:
                l = self.left._is_bst(left, self.value)
            if self.right:
                r = self.right._is_bst(self.value, right)
            return l and r
        else:
            return True
        
class BinaryTree(object):
    def __init__(self):
        self.root = None
        
    def add_node(self, value):
        if not self.root:
            self.root = NodeBT(value)
        else:
            self.root._add_next_node(value)
            
    def is_leaf(self, value):
        node = self.root._search_for_node(value)
        if node:
            return node._is_leaf()
        
        else:
            return False
        
    def get_node_level(self, value):
        node = self.root._search_for_node(value)
        if node:
            return node.level
        
        else:
            return False
        
    def is_root(self, value):
        return self.root.value == value
    
    def get_height(self):
        return self.root._get_max_height()
    
    def is_balanced(self):
        return self.root._is_balanced()
    
    def is_bst(self):
        return self.root._is_bst()

In [10]:
class NodeBST(NodeBT):
    # 위의 NodeBT에서 상속
    def __init__(self, value=None, level=1):
        self.value = value
        self.level = level
        self.left = None
        self.right = None
        
    def _add_next_node(self, value, level_here=2):
        new_node = NodeBST(value, level_here)
        if value > self.value:
            self.right = self.right and self.right._add_next_node(value, level_here + 1) or new_node
        elif value < self.value:
            self.left = self.left and self.left._add_next_node(value, level_here + 1) or new_node
        else:
            print('중복 노드를 허용하지 않습니다.')
        return self
    
    def _search_for_node(self, value):
        if self.value == value:
            return self
        elif self.left and value < self.value:
            return self.left._search_for_node(value)
        elif self.right and value > self.value:
            return self.right._search_for_node(value)
        else:
            return False
        

class BinarySearchTree(BinaryTree):
    # 위의 BinaryTree에서 상속
    def __init__(self):
        self.root = None
        
    def add_node(self, value):
        if not self.root:
            self.root = NodeBST(value)
        else:
            self.root._add_next_node(value)

###### Ch 14 이진 트리

**순회**<sub>traversal</sub>는 트리 또는 그래프 같은 연결된 구조에서 객체(노드)를 방문하는 데 사용되는 알고리즘이다. 순회 문제는 모든 노드를 방문하는 방법을 찾거나 혹은 특정 노드만 방문하는 방법을 찾는 것일 수도 있다. 

# 14.1 깊이 우선 탐색 

**깊이 우선 탐색**<sub>depth-first search</sub> (DFS)은 말 그대로 그래프 또는 트리에서 깊이를 우선하여 탐색하는 알고리즘이다.  그래프의 경우는 방문한 노드를 표시해야 하는데, 그렇게 하지 않으면 무한 반복에 빠질 수 있기 때문이다. 시간 복잡도는 $$O(도달할 수 있는 노드수 + 도달한 노드에서 나가는 간선 수) = O(V + E)$$이다.
깊이 우선 탐색은 후입선출(LIFO) 구조의 스택을 사용하여 구현된다. 

깊이 우선 탐색의 세 가지 경우를 살펴보자. 
1. 전위 순회
2. 후위 순회
3. 중위 순회

## 14.1.1 전위 순회

**전위 순회**<sub>pre-order traversal</sub>는 루트 노드 → 왼쪽 노드 → 오른쪽 노드 순으로 방문한다. 

<img src='https://slid-capture.s3.ap-northeast-2.amazonaws.com/public/image_upload/d4954708fd8f4731a091c75bea2b2ea7/05c8fa97-397f-4b02-b7c4-0f2004f6883d.png' width='300' />

In [3]:
# class BinBinarySearchTree(BianryTree): 에 추가해준다. 

def preorder(root):
    if root != 0:
        yield root.value
        preorder(root.left)
        preorder(root.right)

## 14.1.2 후위 순회

**후위 순회**<sub>post-order traversal</sub> 는 왼쪽 노드 → 오른쪽 노드 → 루트 노드 순으로 방문한다. 

<img src='https://slid-capture.s3.ap-northeast-2.amazonaws.com/public/image_upload/d4954708fd8f4731a091c75bea2b2ea7/88650ba1-e061-49bb-9546-663fbfc752ea.png' width='300' />

In [3]:
# class BinBinarySearchTree(BianryTree): 에 추가해준다. 

def postorder(root):
    if root != 0:
        postorder(root.left)
        postorder(root.right)
        yield root.value

## 14.1.3 중위 순회

**중위 순회**<sub>in-order traversal</sub>는 왼쪽 노드 → 루트 노드 → 오른쪽 노드 순으로 방문한다. 

<img src='https://slid-capture.s3.ap-northeast-2.amazonaws.com/public/image_upload/d4954708fd8f4731a091c75bea2b2ea7/52008641-f2c5-4ca8-8890-436dc2b60f07.png' width='300' />

In [4]:
# class BinBinarySearchTree(BianryTree): 에 추가해준다. 

def inorder(root):
    if root != 0:
        inorder(root.left)
        yield root.value        
        inorder(root.right)

# 너비 우선 탐색 

**너비 우선 탐색**<sub>breath-first search</sub>(BFS)은 트리 또는 그래프에서 너비를 우선하여 탐색하는 알고리즘이다. 더 깊은 노드를 순회하기 전에 특정 깊이의 노드를 모두 먼저 순회한다.   

너비 우선 탐색을 사용하는 문제는 일반적으로 시작 노드에서 특정 노드에 도달하는 데 필요한 최단 경로를 찾는 문제다. 너비 우선 탐색의 구현은 방문한 노드를 저장하는 데에 리스트를 사용하고, 아직 방문하지 않은 노드는 선입 선출(FIFO) 구조의 큐에 저장한다. 
 <br>시간복잡도는 $O(V+E)$다.

<img src='https://slid-capture.s3.ap-northeast-2.amazonaws.com/public/image_upload/d4954708fd8f4731a091c75bea2b2ea7/6b1141d7-7d63-4ad4-8d05-455f1dcfcf1d.png' width='300' />

# 트리 순회 구현하기

반복문으로 구현

In [17]:
from collections import deque

class BSTwithTransversalIterative(BinarySearchTree):
    def inorder(self):
        current = self.root
        nodes, stack = [], []
        while stack or current:
            if current:
                stack.append(current)
                current = current.left
            else:
                current = stack.pop()
                nodes.append(current.value)
                current = current.right
        return nodes
    
    def preorder(self):
        current = self.root
        nodes, stack = [], []
        while stack or current:
            if current:
                nodes.append(current.value)
                stack.append(current)
                current = current.left
            else:
                current = stack.pop()
                current = current.right
        return nodes
    
    def preorder2(self):
        nodes = []
        stack = [self.root]
        while stack:
            current = stack.pop()
            if current:
                nodes.append(current.value)
                stack.append(current.right)
                stack.append(current.left)
        return nodes
    
    def BFT(self):
        current = self.root
        nodes = []
        queue = deque()
        queue.append(current)
        while queue:
            current = queue.popleft()
            nodes.append(current.value)
            if current.left:
                queue.append(current.left)
            if current.right:
                queue.append(current.right)
        return nodes
        
    
                

In [18]:
bst = BSTwithTransversalIterative()
l = [10, 5, 6, 3, 8, 2, 1, 11, 9, 4]
for i in l:
    bst.add_node(i)

In [19]:
print('노드 8은 말단 노드입니까?', bst.is_leaf(8))
print('노드 8의 레벨은?', bst.get_node_level(8))
print("노드 10은 루트 노드입니까? ", bst.is_root(10))
print("노드 1은 루트 노드입니까? ", bst.is_root(1))
print("트리의 높이는? ", bst.get_height())
print("이진 탐색 트리입니까? ", bst.is_bst())
print("균형 트리입니까? ", bst.is_balanced())

print("전위 순회: ", bst.preorder())
print("전위 순회2: ", bst.preorder2())
print("중위 순회: ", bst.inorder())
print("너비 우선 탐색: ", bst.BFT())

노드 8은 말단 노드입니까? False
노드 8의 레벨은? 4
노드 10은 루트 노드입니까?  True
노드 1은 루트 노드입니까?  False
트리의 높이는?  4
이진 탐색 트리입니까?  True
균형 트리입니까?  False
전위 순회:  [10, 5, 3, 2, 1, 4, 6, 8, 9, 11]
전위 순회2:  [10, 5, 3, 2, 1, 4, 6, 8, 9, 11]
중위 순회:  [1, 2, 3, 4, 5, 6, 8, 9, 10, 11]
너비 우선 탐색:  [10, 5, 11, 3, 6, 2, 4, 8, 1, 9]


<img src='https://slid-capture.s3.ap-northeast-2.amazonaws.com/public/image_upload/d4954708fd8f4731a091c75bea2b2ea7/94bddcca-518c-4a12-8716-323915ee4420.png' width='400' />

재귀 함수로 구현

In [20]:
class BSTwithTransversalRecursively(BinarySearchTree):

    def __init__(self):
        self.root = None
        self.nodes_BFS = []
        self.nodes_pre = []
        self.nodes_post = []
        self.nodes_in = []

    def BFT(self):
        self.root.level = 1
        queue = [self.root]
        current_level = self.root.level

        while len(queue) > 0:
            current_node = queue.pop(0)
            if current_node.level > current_level:
                current_level += 1
            self.nodes_BFS.append(current_node.value)

            if current_node.left:
                current_node.left.level = current_level + 1
                queue.append(current_node.left)

            if current_node.right:
                current_node.right.level = current_level + 1
                queue.append(current_node.right)

        return self.nodes_BFS

    def inorder(self, node=None, level=1):
        if not node and level == 1:
            node = self.root
        if node:
            self.inorder(node.left,  level+1)
            self.nodes_in.append(node.value)
            self.inorder(node.right, level+1)
        return self.nodes_in

    def preorder(self, node=None, level=1):
        if not node and level == 1:
            node = self.root
        if node:
            self.nodes_pre.append(node.value)
            self.preorder(node.left, level+1)
            self.preorder(node.right, level+1)
        return self.nodes_pre

    def postorder(self, node=None, level=1):
        if not node and level == 1:
            node = self.root
        if node:
            self.postorder(node.left, level+1)
            self.postorder(node.right, level+1)
            self.nodes_post.append(node.value)
        return self.nodes_post

In [21]:

    bst = BSTwithTransversalRecursively()
    l = [10, 5, 6, 3, 8, 2, 1, 11, 9, 4]
    for i in l:
        bst.add_node(i)

    print("노드 8은 말단 노드입니까? ", bst.is_leaf(8))
    print("노드 8의 레벨은? ", bst.get_node_level(8))
    print("노드 10은 루트 노드입니까? ", bst.is_root(10))
    print("노드 1은 루트 노드입니까? ", bst.is_root(1))
    print("트리의 높이는? ", bst.get_height())
    print("이진 탐색 트리입니까? ", bst.is_bst())
    print("균형 트리입니까? ", bst.is_balanced())

    print("전위 순회: ", bst.preorder())
    print("후위 순회: ", bst.postorder())
    print("중위 순회: ", bst.inorder())
    print("너비 우선 탐색: ", bst.BFT())

노드 8은 말단 노드입니까?  False
노드 8의 레벨은?  4
노드 10은 루트 노드입니까?  True
노드 1은 루트 노드입니까?  False
트리의 높이는?  4
이진 탐색 트리입니까?  True
균형 트리입니까?  False
전위 순회:  [10, 5, 3, 2, 1, 4, 6, 8, 9, 11]
후위 순회:  [1, 2, 4, 3, 9, 8, 6, 5, 11, 10]
중위 순회:  [1, 2, 3, 4, 5, 6, 8, 9, 10, 11]
너비 우선 탐색:  [10, 5, 11, 3, 6, 2, 4, 8, 1, 9]


# 연습문제

두 노드의 최소 공통 조상<sub>lowest common ancestor</sub>을 찾아 보자 

<img src='https://slid-capture.s3.ap-northeast-2.amazonaws.com/public/image_upload/d4954708fd8f4731a091c75bea2b2ea7/de1271a8-9d79-4f3e-94a8-fafe9daef245.png' width='400' />

4, 9의 최소 공통 조상은 5이고, 6, 8의 최소 공통 조상은 6이다. 

In [23]:
def find_ancestor(path, low_value, high_value):
    # 두 인수는 트리에 존재하는 노드여야 하며 두 번째 인수가 더 커야 한다.
    while path:
        current_value = path[0]
        if current_value < low_value:
            try:
                path = path[2:]
            except:
                return current_value
        elif current_value > high_value:
            try:
                path = path[1:]
            except:
                return current_value
        elif low_value <= current_value <= high_value:
            return current_value

In [27]:
    bst = BSTwithTransversalRecursively()
    l = [10, 5, 6, 3, 8, 2, 1, 11, 9, 4]
    for i in l:
        bst.add_node(i)
    path = bst.preorder()
    print("전위 순회: ", path)


    # 입력된 두 노드(인자)는 트리에 존재하며 오름차순으로 입력한다.
    print("1과 6의 공통 상위 조상 :", find_ancestor(path, 1, 6))
    print("1과 11의 공통 상위 조상: ", find_ancestor(path, 1, 11))
    print("1과 4의 공통 상위 조상: ", find_ancestor(path, 1, 4))
    print("8와 9의 공통 상위 조상: ", find_ancestor(path, 8, 9))
    print('##########################')
    print("1과 6의 최소 공통 조상 :", find_ancestor(path, 1, 6))
    print("1과 11의 최소 공통 조상: ", find_ancestor(path, 1, 11))
    print("1과 4의 최소 공통 조상: ", find_ancestor(path, 1, 4))
    print("8와 9의 최소 공통 조상: ", find_ancestor(path, 8, 9))

전위 순회:  [10, 5, 3, 2, 1, 4, 6, 8, 9, 11]
1과 6의 공통 상위 조상 : 5
1과 11의 공통 상위 조상:  10
1과 4의 공통 상위 조상:  3
8와 9의 공통 상위 조상:  8
##########################
1과 6의 최소 공통 조상 : 5
1과 11의 최소 공통 조상:  10
1과 4의 최소 공통 조상:  3
8와 9의 최소 공통 조상:  8


이 책을 끝내며

긱포긱 : www.geeksforgeeks.org

**Reference**

* <a href='https://github.com/SeWonKwon' ><div> <img src ='https://slid-capture.s3.ap-northeast-2.amazonaws.com/public/image_upload/6556674324ed41a289a354258718280d/964e5a8b-75ad-41fc-ae75-0ca66d06fbc7.png' align='left' /> </div></a>

<br>

* [파이썬 자료구조와 알고리즘, 미아 스타인](https://github.com/AstinCHOI/Python-and-Algorithms-and-Data-Structures)
